In [1]:
root = "/home/saint/"
DIRECTORY        = root + "datasets/competitions/tgs-salt-identification-challenge"
SAVE_PATH        = root + "models/salt/"
SUBMISSIONS_PATH = root + "submissions/salt/"
WEIGHTS_PATH     = root + "weights/ternaus_net_v2_deepglobe_buildings.pt"

MODEL_NAME = 'ternausv2'

In [2]:
import torch
torch.__version__

'0.5.0a0+5eb9d40'

In [3]:
# Debug
from IPython.core.debugger import set_trace

# Lenin straight from repo
import sys
sys.path.insert(0, "/home/saint/berloga-dl/lenin")
import lenin
from lenin import train, test
from lenin.datasets.salt import Dataset

# Ternaus net from our repo
sys.path.insert(0, "/home/saint/berloga-dl/salt")
from src.components.TernausNetV2 import *
from src.utils import *

# Torch and torchbearer
import torch
from torch import nn
from torch.nn import functional as F
from torchbearer.callbacks.checkpointers import Best
from torch.autograd import Variable

# stdlib
import json
import copy
import string
import random
from time import gmtime, strftime
from collections import OrderedDict
import numpy as np
import pandas as pd

# Plots
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")


def load_model(model_path):
    model = get_model(3)
    state = torch.load(model_path)
    model.load_state_dict(state['model'])
    model.eval()
    return model

In [4]:
class TernausNetV2(nn.Module):
    fields = ('image', 'mask')
    test_fields = ('image',)
    
    def __init__(self, num_classes=2,
                       num_filters=32,
                       is_deconv=False,
                       num_input_channels=11):
        super().__init__()
        conf = {
             "network": {
                "arch": "wider_resnet38",
                "activation": "leaky_relu",
                "leaky_relu_slope": 0.01,
                "input_3x3": True,
                "bn_mode": "inplace",
                "classes": 1000
            }
        }
        

        self.pool = nn.MaxPool2d(2, 2)

        model_params = get_model_params(conf["network"])

        encoder = WiderResNet(structure=[3, 3, 6, 3, 1, 1], **model_params)

        self.conv1 = nn.Sequential(
            OrderedDict([('conv1', nn.Conv2d(num_input_channels, 64, 3, padding=1, bias=False))]))
        self.conv2 = encoder.mod2
        self.conv3 = encoder.mod3
        self.conv4 = encoder.mod4
        self.conv5 = encoder.mod5
        
        dec_size = 1024
        self.center = DecoderBlock(dec_size, num_filters * 8, num_filters * 8, is_deconv=is_deconv)
        self.dec5 = DecoderBlock(dec_size + num_filters * 8, num_filters * 8, num_filters * 8, is_deconv=is_deconv)
        self.dec4 = DecoderBlock(dec_size//2 + num_filters * 8, num_filters * 8, num_filters * 8, is_deconv=is_deconv)
        self.dec3 = DecoderBlock(dec_size//4 + num_filters * 8, num_filters * 2, num_filters * 2, is_deconv=is_deconv)
        self.dec2 = DecoderBlock(dec_size//8 + num_filters * 2, num_filters * 2, num_filters, is_deconv=is_deconv)
        self.dec1 = ConvRelu(dec_size//16 + num_filters, num_filters)
        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)
        
        
    def freeze(self, layer_name):
        for param in getattr(self, layer_name).parameters():
            param.requires_grad = False
            
            
    def unfreeze(self, layer_name):
        if layer_name == 'all':
            for param in self.parameters():
                param.requires_grad = True
        else:
            for param in getattr(self, layer_name).parameters():
                param.requires_grad = True
        
        
    def forward(self, batch):
        x = batch.image
        conv1 = self.conv1(x)
        conv2 = self.conv2(self.pool(conv1))
        conv3 = self.conv3(self.pool(conv2))
        conv4 = self.conv4(self.pool(conv3))
        conv5 = self.conv5(self.pool(conv4))

        center = self.center(self.pool(conv5))

        dec5 = self.dec5(torch.cat([center, conv5], 1))

        dec4 = self.dec4(torch.cat([dec5, conv4], 1))
        dec3 = self.dec3(torch.cat([dec4, conv3], 1))
        dec2 = self.dec2(torch.cat([dec3, conv2], 1))
        dec1 = self.dec1(torch.cat([dec2, conv1], 1))
        final = self.final(dec1)
        return torch.sigmoid(final)


def get_model(num_input_channels, load_weights=True):
    model = TernausNetV2(num_classes=2)
    if load_weights:
        state = torch.load(WEIGHTS_PATH)
        state = {key.replace('module.', ''): value for key, value in state['model'].items()}

        model.load_state_dict(state)
        model.eval()
    
    model.conv1 = nn.Sequential(
        nn.Conv2d(num_input_channels, 11, 1, padding=0, bias=False), model.conv1)
    
    if torch.cuda.is_available():
        model.cuda()
    return model

In [5]:
# Pre/postprocessors
from skimage.transform import resize
def upsample(img):
    #img = resize(img, (128, 128), mode='constant', preserve_range=True)
    img = pad(image=img)['image']
    return img

def downsample(img):
    #img = resize(img, (101, 101), mode='constant', preserve_range=True)
    img = crop(image=img)['image']
    return img

import albumentations as aug
pad = aug.PadIfNeeded(min_height=128, min_width=128)
crop = aug.CenterCrop(height=101, width=101)
imgs_mean = 0.469914
imgs_std = 0.163075
cumsums_mean = 0.23724
cumsums_std = 0.15206
depths_mean = 0.52639
depths_std = 0.214675
# First channel – color. Second – cumsum. Third – depth with scaling
#norm = aug.Normalize(mean=(imgs_mean,cumsums_mean,depths_mean), std=(imgs_std,cumsums_std,depths_std))
#norm = aug.Normalize(mean=(imgs_mean,imgs_mean,imgs_mean), std=(imgs_std,imgs_std,imgs_std))
norm = aug.Normalize(mean=(0.485,0.485,0.485), std=(0.229,0.229,0.229))

def depths_channel(depth, dim):
    depths_scaling = 0.1
    depths_max = 959. + 128. * depths_scaling # 128 – height of scaled image

    depths_arr = np.cumsum(np.ones(dim) * depths_scaling, axis=0) + depth
    depths_arr = (depths_arr / depths_max) * 255.
    return depths_arr

def cumsum_channel(channel):
    cumsums_max = 25755.

    cumsum_arr = np.cumsum(channel, axis=0)
    cumsum_arr = (cumsum_arr / cumsums_max) * 255.
    return cumsum_arr
    
def img_preprocess(img):
    img = upsample(img)
    
    # Add cumsum of img:
    #img[:, :, 1] = cumsum_channel(img[:, :, 0])
    # Add depth of img
    # img[:, :, 2] = depths_channel()
    img = norm(image=img)['image']
    return img

def img_postprocess(img):
    img = np.transpose(img, (2, 0, 1)).astype('float32')
    return img

def mask_preprocess(mask):
    mask = upsample(mask)
    return mask
    
def mask_postprocess(mask):
    ones = np.expand_dims(mask, 2) > 0
    zeros = ones != True
    mask = np.concatenate((ones, zeros), axis=2)
    mask = np.transpose(mask, (2, 0, 1)).astype('float32')
    return mask

In [6]:
dataset = Dataset(DIRECTORY)
#dataset.check_integrity() NOT WORKING
dataset.preprocessors = { 'image': img_preprocess, 'mask': mask_preprocess }
dataset.postprocessors = { 'image': img_postprocess, 'mask': mask_postprocess }
#show_images(dataset)

In [7]:
def castF(x):
    return x.astype(float)

def castB(x):
    return x.astype(bool)

def iou_loss_core(true, pred):
    intersection = true * pred
    notTrue = 1 - true
    union = true + (notTrue * pred)
    i = np.sum(intersection, axis=-1) + 1e-07
    u = np.sum(union, axis=-1) + 1e-07
    return i / u

def iou_ver2(true, pred):
    tresholds = [0.5 + (i*.05)  for i in range(10)]

    # flattened images (batch, pixels)
    true.shape = (true.shape[0], -1)
    pred.shape = (pred.shape[0], -1)
    pred = castF(pred > 0.5)
    
    # total white pixels - (batch,)
    trueSum = np.sum(true, axis=-1)
    predSum = np.sum(pred, axis=-1)
    
    # has mask or not per image (batch,)
    true1 = castF(trueSum > 1)
    pred1 = castF(predSum > 1)
    
    # to get images that have mask in both true and pred
    truePositiveMask = castB(true1 * pred1)

    #separating only the possible true positives to check iou
    testTrue = true[truePositiveMask]
    testPred = pred[truePositiveMask]

    #getting iou and threshold comparisons
    iou = iou_loss_core(testTrue, testPred) 
    truePositives = [castF(iou > tres) for tres in tresholds]

    #mean of thressholds for true positives and total sum
    truePositives = np.mean(np.stack(truePositives, axis=-1), axis=-1)
    truePositives = np.sum(truePositives)

    #to get images that don't have mask in both true and pred
    trueNegatives = (1 - true1) * (1 - pred1) # = 1 -true1 - pred1 + true1*pred1
    trueNegatives = np.sum(trueNegatives) 

    return (truePositives + trueNegatives) / float(true.shape[0])

In [8]:
def train_step(model, dataset, step_index=0, model_name='', **options):
    save_filepath=SAVE_PATH + ('step%i_%s.{epoch:02d}-{val_loss:.4f}.pt' % (step_index, model_name))
    
    print("Train step %i: %s" % (step_index, save_filepath))
    split = options.pop('split', None)
    if split:
        options['split'] = type(split)
    printable_options = "\n".join([("%s: %s" % (str(k).ljust(12), v)) for k, v in options.items()])
    print("Options:\n%s" % printable_options)
    if split:
        options['split'] = split
    
    checkpointer = Best(filepath=save_filepath)
    options['callbacks'] = [checkpointer]
    
    [model.freeze(layer_name) for layer_name in options.pop('freeze', [])]
    [model.unfreeze(layer_name) for layer_name in options.pop('unfreeze', [])]
    
    train(model, dataset, **options)


from sklearn.model_selection import train_test_split
def split_data(dataset):
    split = {}
    if dataset.__dict__.get('stratify'):
        split['stratify'] = [getattr(dataset, dataset.stratify)(record) for record in dataset.train]
    return tuple(train_test_split(dataset.train, **split))
    
def train_sequence(dataset, model=None, steps=[], **options):
    if not model:
        model = get_model(3, load_weights=True)
        timed_model_name = id_generator(MODEL_NAME)
        print(timed_model_name)
    
    options['split'] = split_data(dataset)
    for i, step in enumerate(steps):
        for k, v in step.items():
            options[k] = v
        opts = copy.deepcopy(options)
        train_step(model, dataset, step_index=i, model_name=timed_model_name, **opts)

from torchbearer import metrics

@metrics.default_for_key('acc')
@metrics.lambda_metric('iou_metric_tb', on_epoch=True)
def iou_metric_tb(y_pred, y_true):
    pred = np.array([downsample(pr) for pr in y_pred.data.cpu().numpy()[:, 0, :, :]])
    true = np.array([downsample(tr) for tr in y_true.data.cpu().numpy()[:, 0, :, :]])
    return { 'running_iou': iou_ver2(true, pred) }
    return iou_ver2(true, pred)

TRAIN = True#False#
if False:#TRAIN:
    model = get_model(3, load_weights=True)
    timed_model_name = id_generator(MODEL_NAME)
    print(timed_model_name)  

In [9]:
if TRAIN:
    options = {
        'augment': { ('image', 'mask'): [
            #{'type': 'PadIfNeeded', 'min_height': 128, 'min_width': 202},
            #{'type': 'RandomCrop', 'height': 128, 'width': 128},
            #{'type': 'HorizontalFlip'},
            #{'type': 'Blur'},
        ] },
        'batch_size': 30,
        'optimizer': ('adam', { 'lr': 1e-4 }),
        'epochs': 12,
        'folds': 5,
        'loss': 'bce',
        'metrics': ['loss'] #, 'acc']
    }
    
    FOLDS = False#True#
    STEPS = True
    if STEPS:
        # One step
        # loss: BCE
        #   No aug: 0.169-0.17
        #   HorizontalFlip: 0.146-0.149
        #   Pad(Horizontally to 202)+RandCrop: 0.151-0.155
        #   HorizontalFlip+Pad(Horizontally to 202)+RandCrop: 0.137-0.142-0.166 -- 0.144-0.152 (0.763 LB)
        if True:
            steps = [
                {}
            ]
            train_sequence(dataset, steps=steps, **options)
        # One step
        # loss: DICE
        #   No aug: 0.0473-0.0479 (0.775 LB)
        #   HorizontalFlip: 
        #   Pad(Horizontally to 202)+RandCrop: 0.0588
        elif False:
            steps = [
                {}
            ]
            options['loss'] = 'dice'
            train_sequence(dataset, steps=steps, **options)
        # One step
        # loss: BCE WITH DICE
        #   No aug: 0.2263 / 0.207
        #   HorizontalFlip: 0.2078 (0.782 LB)
        #   Pad(Horizontally to 202)+RandCrop: 0.161-0.174 (overfit - 0.77 LB)
        #   Blur: 0.236
        #   Normalization by images (all 3 channels identic):
        #     No aug:
        elif False:
            steps = [
                {}
            ]
            options['loss'] = 'bce_with_dice'
            train_sequence(dataset, steps=steps, **options)
        # Freezing all except conv1 and dec1, final
        # Decreasing LR and unfreezing form sides to center
        # loss: BCE
        #   No aug: 0.19-0.191
        #   HorizontalFlip: 0.163-0.164
        elif False:
            steps = [
                {
                    'epochs': 2,
                    'optimizer': ('adam', { 'lr': 1e-3 }),
                    'freeze': ['conv2', 'conv3', 'conv4', 'conv5', 'center', 'dec5', 'dec4', 'dec3', 'dec2']
                },
                {
                    'optimizer': ('adam', { 'lr': 0.5e-3 }),
                    'freeze': [],
                    'unfreeze': ['conv2', 'dec2']
                },
                {
                    'optimizer': ('adam', { 'lr': 1e-4 }),
                    'unfreeze': ['conv3', 'dec3']
                },
                {
                    'optimizer': ('adam', { 'lr': 0.5e-4 }),
                    'unfreeze': ['conv4', 'dec4']
                },
                {
                    'epochs': 5,
                    'optimizer': ('adam', { 'lr': 1e-5 }),
                    'unfreeze': ['conv5', 'center', 'dec5']
                }
            ]
            train_sequence(dataset, steps=steps, **options)
        # BCE
        # Freezing all except conv1 and dec1, final
        # Increasing LR and unfreezing form sides to center
        # loss: BCE
        #   No aug: 0.171-0.18
        elif False: 
            steps = [
                {
                    'epochs': 2,
                    'optimizer': ('adam', { 'lr': 1e-5 }),
                    'freeze': ['conv2', 'conv3', 'conv4', 'conv5', 'center', 'dec5', 'dec4', 'dec3', 'dec2']
                },
                {
                    'optimizer': ('adam', { 'lr': 0.5e-4 }),
                    'freeze': [],
                    'unfreeze': ['conv2', 'dec2']
                },
                {
                    'optimizer': ('adam', { 'lr': 1e-4 }),
                    'unfreeze': ['conv3', 'dec3']
                },
                {
                    'optimizer': ('adam', { 'lr': 0.5e-3 }),
                    'unfreeze': ['conv4', 'dec4']
                },
                {
                    'epochs': 5,
                    'optimizer': ('adam', { 'lr': 1e-3 }),
                    'unfreeze': ['conv5', 'center', 'dec5']
                }   
            ]
            train_sequence(dataset, steps=steps, **options)
        # Freezing all except conv1 and dec1, final
        # Restart LR in every step, unfreezing form sides to center
        # loss: BCE
        #   No aug: 0.178-0.19
        elif False:
            steps = [
                {
                    'epochs': 2,
                    'optimizer': ('adam', { 'lr': 1e-4 }),
                    'freeze': ['conv2', 'conv3', 'conv4', 'conv5', 'center', 'dec5', 'dec4', 'dec3', 'dec2']
                },
                {
                    'freeze': [],
                    'unfreeze': ['conv2', 'dec2']
                },
                {
                    'unfreeze': ['conv3', 'dec3']
                },
                {
                    'unfreeze': ['conv4', 'dec4']
                },
                {
                    'epochs': 5,
                    'unfreeze': ['conv5', 'center', 'dec5']
                }
            ]
            train_sequence(dataset, steps=steps, **options)
        # Freezing all except center, final
        # Decreasing LR and unfreezing form center to sides
        # loss: BCE
        #   No aug: 0.167-0.174
        elif False:
            steps = [
                {
                    'epochs': 2,
                    'optimizer': ('adam', { 'lr': 1e-3 }),
                    'freeze': ['conv1', 'conv2', 'conv3', 'conv4', 'conv5', 'dec5', 'dec4', 'dec3', 'dec2', 'dec1']
                },
                {
                    'freeze': [],
                    'optimizer': ('adam', { 'lr': 0.5e-3 }),
                    'unfreeze': ['conv5', 'dec5']
                },
                {
                    'optimizer': ('adam', { 'lr': 1e-4 }),
                    'unfreeze': ['conv4', 'dec4']
                },
                {
                    'optimizer': ('adam', { 'lr': 0.5e-4 }),
                    'unfreeze': ['conv3', 'dec3']
                },
                {
                    'epochs': 5,
                    'optimizer': ('adam', { 'lr': 1e-5 }),
                    'unfreeze': ['conv1', 'conv2', 'dec2', 'dec1']
                }
            ]
            train_sequence(dataset, steps=steps, **options)
    elif FOLDS:
        # Trying folds
        import copy
        from sklearn.model_selection import KFold, StratifiedKFold

        nets = []
        folds = options.pop('folds', 3)

        if dataset.stratify:
            stratify = [getattr(dataset, dataset.stratify)(record) for record in dataset.train]
            splits = StratifiedKFold(n_splits=folds).split(dataset.train, stratify)
        else:
            splits = KFold(n_splits=folds).split(dataset.train)

        all_records = np.array(dataset.train)
        for i, (train_ids, valid_ids) in enumerate(splits):
            print('Fold %i' % i)
            train_records = all_records[train_ids]
            valid_records = all_records[valid_ids]

            options['split'] = (train_records, valid_records)
            fold_net = copy.deepcopy(model)
            save_filepath=SAVE_PATH + ('f%i' % i) + ('%s.{epoch:02d}-{val_loss:.4f}.pt' % timed_model_name)
            checkpointer = Best(filepath=save_filepath)
            options['callbacks'] = [checkpointer]
            train(fold_net, dataset, **options)
            nets.append(fold_net)
    else:
        train_step(model, dataset, step_index=0, **options)

/usr/local/lib/python3.5/dist-packages/torchbearer/torchbearer.py:27: UserWarning: The Model class and all of its methods will be deprecated in the next version (0.2.0) in favor of the upcoming Trial API
  warnings.warn('The Model class and all of its methods will be deprecated in the next version (0.2.0) in favor of the upcoming Trial API')
0/12(t):   0%|          | 0/100 [00:00<?, ?it/s]

2018-09-24-234128_da2b372_ternausv2_st%i_{epoch:02d}_{val_loss:.4f}.pt
Train step 0: /home/saint/models/salt/step0_2018-09-24-234128_da2b372_ternausv2_st%i_{epoch:02d}_{val_loss:.4f}.pt.{epoch:02d}-{val_loss:.4f}.pt
Options:
augment     : {('image', 'mask'): []}
optimizer   : ('adam', {'lr': 0.0001})
folds       : 5
loss        : bce
split       : <class 'tuple'>
metrics     : ['loss']
batch_size  : 30
epochs      : 12


/usr/local/lib/python3.5/dist-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
0/12(t):   5%|▌         | 5/100 [00:04<01:30,  1.06it/s, running_loss=5.42]Process Process-8:
Process Process-4:
Process Process-7:
Process Process-1:
Process Process-3:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _boo

KeyboardInterrupt
  File "/usr/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
%debug

# Inference

In [ ]:
!ls -la {SAVE_PATH}

In [ ]:
TRAIN = False
if not TRAIN:
    model_name = 'step0_21-09-2018_00:02:55_ternausv2.11-0.2455.pt'
    model = load_model(SAVE_PATH + model_name)

In [ ]:
img_id = 'train-5b7c160d0d'
orig_img = dataset.image(img_id)
orig_mask = dataset.mask(img_id)
img = dataset.postprocessors['image'](dataset.preprocessors['image'](orig_img))
img = np.expand_dims(img, 0)
img = torch.from_numpy(img)

batch = TestBatch()
batch.image = torch.autograd.Variable(img).cuda()
pred = model(batch).data[0].cpu().numpy()[0]
assert pred.shape, (128, 128)
pred = downsample(pred)

plt.figure(figsize=(15,10))
plt.subplot(1, 4, 1)
plt.imshow(orig_img)
plt.subplot(1, 4, 2)
plt.imshow(orig_mask)
plt.subplot(1, 4, 3)
plt.imshow(pred)
plt.subplot(1, 4, 4)
plt.imshow(edges(pred, 0.5))

In [ ]:
show_images(dataset, model)

In [ ]:
# src: https://www.kaggle.com/aglotero/another-iou-metric
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)

In [ ]:
val_dataset = Dataset(DIRECTORY)
val_dataset.preprocessors = { 'image': img_preprocess, 'mask': mask_preprocess }
val_dataset.postprocessors = { 'image': img_postprocess, 'mask': mask_postprocess }
val_dataset.test = dataset.train[:2000]
val_predictions = test(model, val_dataset,
    augment={ ('image',): [{'type': 'PadIfNeeded', 'min_height': 128, 'min_width': 128}] }
)

In [ ]:
val_preds = np.array([downsample(pred) for pred in val_predictions.data.cpu().numpy()[:, 0, :, :]])
val_masks = np.array([downsample(val_dataset.mask(rec)) for rec in val_dataset.test])

In [ ]:
val_preds.shape = (2000, -1)
val_preds.shape

In [ ]:
iou_ver2(val_masks, val_preds)

In [ ]:
show_images(dataset, model, threshold=threshold_best)

In [ ]:
threshold_best = 0.5

predictions = test(model, dataset, batch_size=25)
predictions = np.array([downsample(pred) for pred in predictions.data.cpu().numpy()[:, 0, :, :]])
binary_predictions = (predictions > threshold_best).astype(int)

def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

all_masks = []
for p_mask in list(binary_predictions):
    p_mask = rle_encoding(p_mask)
    all_masks.append(' '.join(map(str, p_mask)))

submit = pd.DataFrame([[rec.split('-')[1] for rec in dataset.test], all_masks]).T
submit.columns = ['id', 'rle_mask']
submit.to_csv('/home/saint/submissions/salt/%s.csv' % model_name, index = False)

In [ ]:
submit = SUBMISSIONS_PATH + model_name + '.csv'
!kaggle c submit -f {submit.replace(':', '\:')} -m '{submit}' -c tgs-salt-identification-challenge

In [ ]:
print(submit)